In [6]:
import os
import pandas as pd
from torch.utils.data.dataset import Dataset
import numpy as np
import math

class BrainReaderDataset(Dataset):
    def __init__(self, data_folder, mapping_csv):
        self.data_folder = data_folder
        self.mapping_df = pd.read_csv(mapping_csv)

    def __len__(self):
        return len(self.mapping_df)

    def __getitem__(self, idx):
        file_name = self.mapping_df.iloc[idx, 0]  # Assuming the file names are in the first column
        score = self.mapping_df.iloc[idx, 1]  # Assuming the scores are in the second column

        # Load the data from the CSV file
        data = pd.read_csv(os.path.join(self.data_folder, file_name))

        return data, score 

def custom_collate(batch):
        data_batch, score_batch = zip(*batch)
        # Assuming your data is a Pandas DataFrame, convert it to a NumPy array
        data_batch = [data.values for data in data_batch]
        return data_batch, score_batch


In [12]:
#creating dataset

from torch.utils.data import DataLoader

data_folder = "Kevin"
mapping_csv = "name2score.csv"

dataset = BrainReaderDataset(data_folder, mapping_csv)

#creating a dataloader
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)



In [13]:
# Assuming you have already created the DataLoader
for batch_idx, (data, score) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}:")
    print("Data:")
    print(data)
    print("Scores:")
    print(score)
    print("\n")
    if batch_idx == 0:
        break  # Print only the first batch for brevity

Batch 1:
Data:
[array([[0.70451539, 0.50519323, 1.09326231, 1.72963989, 0.67216612,
        0.55535592, 1.14199488, 1.86497523, 0.633716  , 0.61455874,
        1.15115847, 1.70498846, 0.6311786 , 0.72676251, 1.19764757,
        1.68020101],
       [0.72509873, 0.78145495, 1.26157171, 1.7160258 , 0.81378769,
        0.71256106, 1.3187414 , 1.84491081, 0.93582404, 0.71111259,
        1.33367504, 1.7693272 , 1.07624356, 0.6830654 , 1.37536283,
        1.81319216],
       [1.19354955, 0.67073206, 1.4023828 , 1.80161801, 1.13883281,
        0.65143194, 1.40038807, 1.899171  , 1.15046404, 0.70196266,
        1.35986848, 1.76155465, 1.15827064, 0.76005824, 1.36263901,
        1.71794327],
       [1.1405375 , 0.74534865, 1.33761532, 1.69966323, 1.06651888,
        0.70085187, 1.33097191, 1.82831961, 1.09739176, 0.68782822,
        1.30174991, 1.72168529, 1.09068937, 0.66152257, 1.30230673,
        1.73635017],
       [1.0896971 , 0.63761643, 1.3149498 , 1.75337767, 1.00287953,
        0.637423

In [19]:
import torch
import torch.nn as nn

class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        
        # Define layers
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

# Instantiate the model
input_size = 9 * 16  # Assuming input is a 10x16 array
model = BinaryClassifier(input_size)

# Print the model architecture
print(model)


BinaryClassifier(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=144, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Assuming you have the BinaryClassifier model and DataLoader already defined
input_size = 9 * 16  # Assuming input is a 10x16 array
model = BinaryClassifier(input_size)
data_folder = "Kevin"
mapping_csv = "name2score.csv"
dataset = BrainReaderDataset(data_folder, mapping_csv)
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0.0
    correct_predictions = 0

    for batch_idx, (data, labels) in enumerate(dataloader):
        optimizer.zero_grad()  # Zero the gradients
        print(torch.Tensor(data).shape)
        # Forward pass
        outputs = model(torch.Tensor(data))  # Assuming data is a NumPy array

        # Calculate loss
        labels = torch.Tensor(labels).view(-1, 1)  # Reshape labels to match the output shape
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

        # Accumulate statistics
        total_loss += loss.item()
        predictions = (outputs > 0.5).float()  # Convert probabilities to binary predictions
        correct_predictions += torch.sum(predictions == labels).item()

    # Print statistics for the epoch
    average_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / len(dataset)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}")


torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([1, 9, 16])
Epoch 1/10, Loss: 0.7074, Accuracy: 0.3061
torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([1, 9, 16])
Epoch 2/10, Loss: 0.6970, Accuracy: 0.6939
torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([1, 9, 16])
Epoch 3/10, Loss: 0.6153, Accuracy: 0.6939
torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([1, 9, 16])
Epoch 4/10, Loss: 0.5514, Accuracy: 0.6939
torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([1, 9, 16])
Epoch 5/10, Loss: 0.6972, Accuracy: 0.6939
torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([1, 9, 16])
Epoch 6/10, Loss: 0.5675, Accuracy: 0.6939
torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([16, 9, 16])
torch.Size([1, 9, 16])
Epoch 7/10, Loss: 0.7089, Accuracy: 0.6939
torch.Size([16, 9, 16])
torch.Size

In [ ]:
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import os 
import pandas as pd
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split

data = CustomDatasetFromCSV(PATH)



## generating indices to split data
train_indices, test_indices, _, _ = train_test_split(
    range(len(data)),
    data.targets,
    stratify=data.targets,
    test_size=0.1,
    random_state=42
)



## generating subsets usin indicies
## Note: data is to be replaced with actual Dataset 
training_data = Subset(data, train_indices)

test_data = Subset(data, test_indices)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")


